# VERIFICAÇÃO DOS DADOS

Importando as bibliotecas pertinentes:

In [1]:
import sqlite3
from sqlite3 import Error

import numpy as np
import pandas as pd

Obtendo os dados:

In [2]:
questionario_exame = pd.read_csv("dados/aluno_questionario_exame.csv", sep=';')

questionario_exame.head()

,escola.nome,aluno.numero_identificacao,aluno.periodo,aluno.turma,aluno.nascimento,aluno.sexo,aluno.raca,questionario.data,questionario.questao_1,questionario.questao_2,...,exame.periodontal_bolsa_2627,exame.periodontal_sangramento_3736,exame.periodontal_calculo_3736,exame.periodontal_bolsa_3736,exame.periodontal_sangramento_31,exame.periodontal_calculo_31,exame.periodontal_bolsa_31,exame.periodontal_sangramento_4647,exame.periodontal_calculo_4647,exame.periodontal_bolsa_4647
0,Escola municipal de T.I. Margarida Gonçalves,7064,3.0,92.01,2003-11-20,1.0,3.0,2018-11-13,3.0,2.0,...,9.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,9.0
1,Escola municipal de T.I. Margarida Gonçalves,7172,3.0,92.01,2004-04-23,1.0,3.0,2018-11-13,3.0,0.0,...,9.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,9.0
2,Escola municipal de T.I. Margarida Gonçalves,7257,3.0,92.01,2003-07-14,1.0,3.0,2018-11-13,3.0,0.0,...,9.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,9.0
3,Escola municipal de T.I. Margarida Gonçalves,7307,3.0,92.01,2004-01-14,3.0,1.0,2018-11-13,3.0,0.0,...,9.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,9.0
4,Escola municipal de T.I. Margarida Gonçalves,7154,3.0,92.01,2004-04-07,1.0,1.0,2018-11-13,4.0,2.0,...,9.0,0.0,0.0,9.0,0.0,0.0,9.0,0.0,0.0,9.0


Incluindo as idades no dataframe utilizando o cálculo `exame.data - aluno.nascimento`:

In [3]:
# Obtém o ano de uma string contendo uma data no formato "YYYY-MM-DD"
# Dados faltantes retornam o ano com maior aplicação dos exames (2019)
def extrair_ano(data):
    return int(data[0:4]) if isinstance(data, str) else 2019

questionario_exame['aluno.idade'] = questionario_exame['exame.data'].apply(extrair_ano) - questionario_exame['aluno.nascimento'].apply(extrair_ano)

questionario_exame['aluno.idade']

0       15
1       14
2       15
3       14
4       14
        ..
1414    15
1415    15
1416    15
1417    16
1418    14
Name: aluno.idade, Length: 1419, dtype: int64

In [4]:
questionario_exame['aluno.idade'].value_counts().head(10)

15     732
14     504
0       67
13      31
16      30
119     20
17       6
18       4
11       2
104      2
Name: aluno.idade, dtype: int64

Corrigindo os dados conflitantes em que o aluno preencheu erroneamente a data do exame, para tal utilizaremos no cálculo da idade o ano majoritário de aplicação do exame. Para tal consideraremos os casos em que:

 - A idade calculada for igual a ;
 - A data presente na coluna `exame.data` for anterior a 2018.
 
Nesses casos o preenchimento dos campos `exame.data` e `aluno.nascimento` foi igual e a data duplicada foi a de nascimento.

In [5]:
ind_conflitantes = questionario_exame[(questionario_exame["aluno.idade"] == 0) & (questionario_exame["exame.data"].apply(extrair_ano) < 2018)].index.to_list()

print(ind_conflitantes)

[96, 102, 158, 193, 232, 291, 334, 599, 609, 705, 706, 711, 713, 716, 717, 722, 723, 727, 728, 732, 735, 736, 737, 892, 893, 953, 954, 959, 965, 1066, 1110, 1196, 1210, 1219, 1303, 1304]


In [6]:
# -1 é o índice da coluna "aluno.idade"

questionario_exame.iloc[ind_conflitantes, -1] = np.ones_like(ind_conflitantes)*2019 - questionario_exame.iloc[ind_conflitantes]["aluno.nascimento"].apply(extrair_ano)

questionario_exame['aluno.idade'].value_counts().head(10)

15     754
14     511
16      35
0       31
13      31
119     20
17       7
18       4
25       2
104      2
Name: aluno.idade, dtype: int64

Verificando as colunas do exame:

In [7]:
colunas_exame = questionario_exame.columns[157:221:2]

print('Colunas referentes ao exame:')
print(colunas_exame)

Colunas referentes ao exame:
Index(['exame.carie_coroa_18', 'exame.carie_coroa_17', 'exame.carie_coroa_16',
       'exame.carie_coroa_15', 'exame.carie_coroa_14', 'exame.carie_coroa_13',
       'exame.carie_coroa_12', 'exame.carie_coroa_11', 'exame.carie_coroa_21',
       'exame.carie_coroa_22', 'exame.carie_coroa_23', 'exame.carie_coroa_24',
       'exame.carie_coroa_25', 'exame.carie_coroa_26', 'exame.carie_coroa_27',
       'exame.carie_coroa_28', 'exame.carie_coroa_38', 'exame.carie_coroa_37',
       'exame.carie_coroa_36', 'exame.carie_coroa_35', 'exame.carie_coroa_34',
       'exame.carie_coroa_33', 'exame.carie_coroa_32', 'exame.carie_coroa_31',
       'exame.carie_coroa_41', 'exame.carie_coroa_42', 'exame.carie_coroa_43',
       'exame.carie_coroa_44', 'exame.carie_coroa_45', 'exame.carie_coroa_46',
       'exame.carie_coroa_47', 'exame.carie_coroa_48'],
      dtype='object')


Incluindo as somas C, P, O e CPO individuais:

In [8]:
# Função auxiliar para verificação de código para cárie dentária

def verificar_C(v):
    if isinstance(v, str) and v.isdigit():
        v = int(v)
    
    return v in [1, 2]

def verificar_P(v):
    if isinstance(v, str) and v.isdigit():
        v = int(v)
    
    return v in [4, 5]

def verificar_O(v):
    if isinstance(v, str) and v.isdigit():
        v = int(v)
    
    return v == 3

In [9]:
questionario_exame['C'] = [sum(questionario_exame[colunas_exame].iloc[i].apply(verificar_C))
                           for i in range(questionario_exame.shape[0])]

questionario_exame['P'] = [sum(questionario_exame[colunas_exame].iloc[i].apply(verificar_P))
                           for i in range(questionario_exame.shape[0])]

questionario_exame['O'] = [sum(questionario_exame[colunas_exame].iloc[i].apply(verificar_O))
                           for i in range(questionario_exame.shape[0])]

questionario_exame['soma_CPO'] = questionario_exame['C'] + questionario_exame['P'] + questionario_exame['O']

questionario_exame.head()

,escola.nome,aluno.numero_identificacao,aluno.periodo,aluno.turma,aluno.nascimento,aluno.sexo,aluno.raca,questionario.data,questionario.questao_1,questionario.questao_2,...,exame.periodontal_calculo_31,exame.periodontal_bolsa_31,exame.periodontal_sangramento_4647,exame.periodontal_calculo_4647,exame.periodontal_bolsa_4647,aluno.idade,C,P,O,soma_CPO
0,Escola municipal de T.I. Margarida Gonçalves,7064,3.0,92.01,2003-11-20,1.0,3.0,2018-11-13,3.0,2.0,...,0.0,9.0,0.0,0.0,9.0,15,0,0,2,2
1,Escola municipal de T.I. Margarida Gonçalves,7172,3.0,92.01,2004-04-23,1.0,3.0,2018-11-13,3.0,0.0,...,0.0,9.0,0.0,0.0,9.0,14,2,0,0,2
2,Escola municipal de T.I. Margarida Gonçalves,7257,3.0,92.01,2003-07-14,1.0,3.0,2018-11-13,3.0,0.0,...,0.0,9.0,0.0,0.0,9.0,15,0,0,0,0
3,Escola municipal de T.I. Margarida Gonçalves,7307,3.0,92.01,2004-01-14,3.0,1.0,2018-11-13,3.0,0.0,...,0.0,9.0,0.0,0.0,9.0,14,0,0,0,0
4,Escola municipal de T.I. Margarida Gonçalves,7154,3.0,92.01,2004-04-07,1.0,1.0,2018-11-13,4.0,2.0,...,0.0,9.0,0.0,0.0,9.0,14,0,0,0,0


# TABELAS DIMENSÃO

## Escolas

In [10]:
escolas = pd.read_csv("dados/lista_escolas.csv", sep=';', decimal=',', encoding='latin-1')

escolas.head()

,escola.nome,escola.latitude,escola.longitude,escola.região,escola.território
0,Escola municipal Luiz Gonzaga,-10.15669,-48.32812,NOROESTE,Kanela
1,Escola municipal Mestre Pacífico S. Campos,-10.15627,-48.34493,NOROESTE,Kanela
2,Escola municipal de T.I. Padre Josimo M. Tavares,-10.17024,-48.33538,NOROESTE,Kanela
3,Escola municipal Anne Frank,-10.17973,-48.31112,NORDESTE,Apinajé
4,Escola municipal Henrique Talone Pinheiro,-10.19480,-48.31044,SUDESTE,Apinajé


Verificando se existe duplicação nos nomes das escolas, advinda de erros de digitação:

In [11]:
escolas['escola.nome'].unique()

array(['Escola municipal Luiz Gonzaga',
       'Escola municipal Mestre Pacífico S. Campos',
       'Escola municipal de T.I. Padre Josimo M. Tavares',
       'Escola municipal Anne Frank',
       'Escola municipal Henrique Talone Pinheiro',
       'Escola municipal Antônio Carlos Jobim',
       'Escola municipal de T.I. Margarida Gonçalves',
       'Escola municipal Antônio G. de Carvalho Filho',
       'Escola municipal Aurélio Buarque de Holanda',
       'Escola municipal de T.I. Aprigio T. de Matos',
       'Escola municipal Crispim Pereira de Alencar',
       'Escola municipal de T.I. Daniel Batista',
       'Escola municipal Darcy Ribeiro', 'Escola municipal Jorge Amado',
       'Escola municipal de T.I. João Beltrão',
       'Escola municipal de T.I. Luiz Nunes de Oliveira',
       'Escola municipal Maria Júlia Amorim Rodrigues',
       'Escola municipal Maria Rosa de Castro Sales',
       'Escola municipal de T.I. Monsenhor Pedro P. Piagem',
       'Escola municipal de T.I. Sue

## Faixa_etária

Verificando as faixas etárias:

In [12]:
for idade in sorted(questionario_exame['aluno.idade'].unique(), reverse=True):
    print(idade, '->', sum(questionario_exame['aluno.idade'] == idade))

7994 -> 1
2014 -> 1
2000 -> 1
929 -> 1
119 -> 20
118 -> 2
104 -> 2
32 -> 1
25 -> 2
19 -> 1
18 -> 4
17 -> 7
16 -> 35
15 -> 754
14 -> 511
13 -> 31
12 -> 1
11 -> 2
4 -> 2
1 -> 1
0 -> 31
-1 -> 2
-12 -> 1
-14 -> 1
-1785 -> 1
-1786 -> 1
-2000 -> 1
-9000 -> 1


In [13]:
faixa_etaria = pd.DataFrame({'idade' : np.arange(12, 20)})             
faixa_etaria

,idade
0,12
1,13
2,14
3,15
4,16
5,17
6,18
7,19


## Exame

Dimensão exame com exame.id único igual a 0:

 - Considerando como representativa a data de maior realização dos exames

In [14]:
data_majoritaria = questionario_exame["exame.data"].value_counts().index[0]

exame = pd.DataFrame({'data' : [data_majoritaria]})

exame

,data
0,2019-06-19


# TABELA FATO

Obtendo informações da granularidade dos dados:

In [15]:
nome_escolas = {escola : index
                for index, escola in escolas['escola.nome'].items()}
idades = {idade : index
          for index, idade in faixa_etaria['idade'].items()}

Gerando a tabela fato:

In [16]:
tabela_fato_data = questionario_exame[['escola.nome', 'aluno.idade', 'C', 'P', 'O', 'soma_CPO']]

tabela_fato_data = tabela_fato_data[tabela_fato_data['aluno.idade'].isin(idades)]  # retirando idades incompatíveis

tabela_fato_data

,escola.nome,aluno.idade,C,P,O,soma_CPO
0,Escola municipal de T.I. Margarida Gonçalves,15,0,0,2,2
1,Escola municipal de T.I. Margarida Gonçalves,14,2,0,0,2
2,Escola municipal de T.I. Margarida Gonçalves,15,0,0,0,0
3,Escola municipal de T.I. Margarida Gonçalves,14,0,0,0,0
4,Escola municipal de T.I. Margarida Gonçalves,14,0,0,0,0
...,...,...,...,...,...,...
1414,Escola municipal Professor Sávia F. Jacome,15,2,0,0,2
1415,Escola municipal Professor Sávia F. Jacome,15,3,0,0,3
1416,Escola municipal Professor Sávia F. Jacome,15,0,0,0,0
1417,Escola municipal Professor Sávia F. Jacome,16,6,0,0,6


In [17]:
somas_group = tabela_fato_data.groupby(by=['escola.nome', 'aluno.idade']).sum()  # obtendo a soma_CPO
contagem_group = tabela_fato_data.groupby(by=['escola.nome', 'aluno.idade'])["soma_CPO"].count()  # obtendo a quantidade da população

In [18]:
tabela_fato = pd.DataFrame({'escola_id' : [nome_escolas[index[0]] for index in somas_group.index],
                            'faixa_etaria_id' : [idades[index[1]] for index in somas_group.index],
                            'C' : [c for c in somas_group['C']],
                            'P' : [p for p in somas_group['P']],
                            'O' : [o for o in somas_group['O']],
                            'soma_cpo' : [cpo for cpo in somas_group['soma_CPO']],
                            'quantidade_populacao' : [pop for pop in contagem_group]})

tabela_fato['exame_id'] = [0] * tabela_fato.shape[0]

tabela_fato

,escola_id,faixa_etaria_id,C,P,O,soma_cpo,quantidade_populacao,exame_id
0,28,1,9,0,0,9,3,0
1,28,2,46,1,39,86,48,0
2,28,3,70,13,52,135,86,0
3,28,4,0,0,3,3,1,0
4,28,5,2,0,1,3,1,0
...,...,...,...,...,...,...,...,...
94,22,3,10,7,10,27,12,0
95,25,1,0,0,0,0,1,0
96,25,2,1,1,11,13,12,0
97,25,3,8,3,13,24,20,0


# CRIAÇÃO DA BASE DE DADOS

## Exportando a base de dados

In [19]:
def create_server_connection(name_database):
    connection = None
    try:
        connection = sqlite3.connect(name_database)
        print("SQLite3 Database created")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [20]:
conn = create_server_connection("observatorio_sorriso")

cur = conn.cursor()

SQLite3 Database created


In [21]:
tabela_fato.to_sql(name='CPO_D', con=conn, if_exists='replace')

99

In [22]:
escolas.to_sql(name='Escola', con=conn, if_exists='replace')

29

In [23]:
faixa_etaria.to_sql(name='Faixa_etaria', con=conn, if_exists='replace')

8

In [24]:
exame.to_sql(name='Exame', con=conn, if_exists='replace')

1

## Data Mart CPO-D

Consultando o SQL criado:

In [25]:
res = cur.execute("SELECT * FROM CPO_D")
res.fetchall()

[(0, 28, 1, 9, 0, 0, 9, 3, 0),
 (1, 28, 2, 46, 1, 39, 86, 48, 0),
 (2, 28, 3, 70, 13, 52, 135, 86, 0),
 (3, 28, 4, 0, 0, 3, 3, 1, 0),
 (4, 28, 5, 2, 0, 1, 3, 1, 0),
 (5, 27, 2, 15, 0, 9, 24, 4, 0),
 (6, 27, 3, 17, 1, 29, 47, 11, 0),
 (7, 27, 5, 0, 0, 0, 0, 1, 0),
 (8, 3, 1, 2, 0, 1, 3, 3, 0),
 (9, 3, 2, 20, 8, 37, 65, 34, 0),
 (10, 3, 3, 15, 0, 29, 44, 35, 0),
 (11, 3, 4, 0, 0, 0, 0, 1, 0),
 (12, 3, 5, 0, 0, 1, 1, 1, 0),
 (13, 5, 1, 0, 0, 0, 0, 1, 0),
 (14, 5, 2, 29, 1, 12, 42, 23, 0),
 (15, 5, 3, 56, 5, 28, 89, 65, 0),
 (16, 7, 1, 0, 0, 2, 2, 1, 0),
 (17, 7, 2, 5, 0, 17, 22, 10, 0),
 (18, 7, 3, 34, 4, 7, 45, 28, 0),
 (19, 8, 2, 18, 1, 13, 32, 22, 0),
 (20, 8, 3, 21, 6, 16, 43, 25, 0),
 (21, 10, 1, 1, 0, 0, 1, 1, 0),
 (22, 10, 2, 17, 0, 23, 40, 14, 0),
 (23, 10, 3, 15, 1, 20, 36, 10, 0),
 (24, 10, 4, 0, 1, 12, 13, 1, 0),
 (25, 10, 5, 6, 0, 1, 7, 1, 0),
 (26, 12, 2, 20, 0, 23, 43, 11, 0),
 (27, 12, 3, 36, 0, 20, 56, 17, 0),
 (28, 12, 4, 0, 0, 0, 0, 1, 0),
 (29, 4, 1, 1, 0, 3, 4, 2, 0),


In [26]:
res = cur.execute("SELECT * FROM Escola")
res.fetchall()

[(0,
  'Escola municipal Luiz Gonzaga',
  -10.15669,
  -48.32812,
  'NOROESTE',
  'Kanela'),
 (1,
  'Escola municipal Mestre Pacífico S. Campos',
  -10.15627,
  -48.34493,
  'NOROESTE',
  'Kanela'),
 (2,
  'Escola municipal de T.I. Padre Josimo M. Tavares',
  -10.17024,
  -48.33538,
  'NOROESTE',
  'Kanela'),
 (3,
  'Escola municipal Anne Frank',
  -10.17973,
  -48.31112,
  'NORDESTE',
  'Apinajé'),
 (4,
  'Escola municipal Henrique Talone Pinheiro',
  -10.1948,
  -48.31044,
  'SUDESTE',
  'Apinajé'),
 (5,
  'Escola municipal Antônio Carlos Jobim',
  -10.2572,
  -48.32097,
  'SUDESTE',
  'Krahô'),
 (6,
  'Escola municipal de T.I. Margarida Gonçalves',
  -10.33109,
  -48.32005,
  'SUDESTE',
  'Xerente'),
 (7,
  'Escola municipal Antônio G. de Carvalho Filho',
  -10.24967,
  -48.3405,
  'SUDOESTE',
  'Xambioá'),
 (8,
  'Escola municipal Aurélio Buarque de Holanda',
  -10.2483,
  -48.3304,
  'SUDESTE',
  'Karajá'),
 (9,
  'Escola municipal de T.I. Aprigio T. de Matos',
  -10.01821,
  -48.

In [27]:
res = cur.execute("SELECT * FROM Faixa_etaria")
res.fetchall()

[(0, 12), (1, 13), (2, 14), (3, 15), (4, 16), (5, 17), (6, 18), (7, 19)]

In [28]:
res = cur.execute("SELECT * FROM Exame")
res.fetchall()

[(0, '2019-06-19')]